In [0]:
'''=============================================================== Data directory ==========================================================
Reading data from path(given from current directory) drive/ML/Inter-IIT-CSRE  (kindly create and place the data in directory like this)'''







#=======================================USE THIS FUNCTION IF LOADING DATA DIRECTLY FROM GOOGLE DRIVE IN GOOGLE COLAB=====================#
'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





!mkdir -p drive
!google-drive-ocamlfuse drive


!ls
'''
#-------------------------------------------------------------------------------------------------------------------------------------------#

In [0]:
# '''==================================================================== Install below libraries =================================================='''
# Link : https://pypi.org/project/tifffile/
#!pip install tifffile 


In [0]:
#'''========================================================== Required IMPORTS ========================================================'''
import os
import time
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

import cv2
import tifffile as tiff

from itertools import product,combinations
from collections import Counter #Link: https://docs.python.org/2/library/collections.html
import h5py #to save model weight
import heapq

#----------------------------------sklearn import----------------------------#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score,accuracy_score


#-----------------------------------Keras imports----------------------------------------------#

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense,Convolution2D, MaxPooling2D, Flatten, Dense,Activation,average,Input
from keras.layers.normalization import BatchNormalization

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from keras import metrics

In [0]:
'''========================================================== Read Image Function ========================================================'''
def reading(path,s): #function to read the data where s is aa name of library used for reading images
  cou_dir_list = sorted(os.listdir(path))
  t = []
  for i in range(len(cou_dir_list)):
    filename = path + "/"+cou_dir_list[i]
    if s == "tiff":
      img = tiff.imread(filename)
    elif s == "cv2":
      img = cv2.imread(filename, 1)
    t.append(img)
  print("Data len:",len(t),"        Shape of 1st ele:",t[0].shape)
  return t
'''----------------------------------------------------------------------------------------------------------------------------------------'''



'''========================================================== scale_percentile Function ========================================================'''
def scale_percentile(matrix):     #Function to convert the given 4 band image to png uint8 image.
    """Fixes the pixel value range to 2%-98% original distribution of values"""
    orig_shape = matrix.shape
    matrix = np.reshape(matrix, [matrix.shape[0]*matrix.shape[1], 3]).astype(float)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    
    matrix = (matrix - mins[None,:])/maxs[None,:]
    matrix = np.reshape(matrix, orig_shape)
    matrix = matrix.clip(0,1)
    return matrix
'''---------------------------------------------------------------------------------------------------------------------------------------'''




'''========================================================== Plot RGB Function ========================================================'''
def plot(img , title): #Function to plot png image
    fig = plt.figure(figsize = (18 , 18))
    ax = fig.subplots(nrows=1, ncols=2)
    ax[0].imshow(img)
    plt.xticks([])
    plt.yticks([]) 
    plt.title (title + ",       Shape:"+ str(img.shape))
# plot(y[0],"Label") #use like this
'''---------------------------------------------------------------------------------------------------------------------------------------'''





'''========================================================== Plot TIF Function ========================================================'''
def plottiff(z,title):#Function to plot .tif format image
  tif_data = z[:,:,:3]
  img = scale_percentile(tif_data)
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([]) 
  plt.title (title + "     Shape:"+ str(img.shape))
  plt.show()
# plottiff(x[0],"Actual") #use like this
'''---------------------------------------------------------------------------------------------------------------------------------------'''

In [0]:
'''###======================================================== READ IMAGES =============================================================='''

x = reading("drive/ML/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/sat","tiff")
y = reading("drive/ML/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/gt","cv2")
test_images = reading("drive/ML/Inter-IIT-CSRE/The-Eye-in-the-Sky-test-data/sat_test","tiff")
somefile = "drive/ML/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/sat/1.tif"
nx = len(x) #no. of train images 
nytest = len(test_images)#no. of test images


In [0]:
'''###============================== DICTIONARY for png colour value from id and viceversa e.g id = 0 and png = (255,255,255)===================================================================###'''

unique = []
for img in y:
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      ele = (img[i,j,0],img[i,j,1],img[i,j,2])
      if ele not in unique:
        unique.append(ele)

pixeltoid = dict(zip(unique, range(9)))
idtopixel = dict(zip(range(9),unique))

'''
[(255, 255, 255), white                  - NOTHING/other class
 (100, 100, 100), gray                   - building
 (0, 0, 0),       black                  - roads
 (0, 255, 0),     light green            - grass
 (0, 125, 0),     dark green             - trees
 (0, 255, 255),   light blue             - railway
 (0, 80, 150),    dark blue              - Bare soil
 (255, 150, 150), light pink             - pool
 (150, 0, 0)]     Brown                  - water

'''


'''###================================================ REPLACE COLOUR WITH ID ================================================================###'''

ymask = []   #2d reprasentaion of image label by replacing png colour with thier id
for ele in y:
  temp = np.zeros((ele.shape[0],ele.shape[1]))
  for i,j in product(range(ele.shape[0]),range(ele.shape[1])):
    temp[i,j] = pixeltoid[tuple(ele[i,j,:])]
  ymask.append(temp)
# print(len(ymask),ymask[0].shape) #use it like this



'''###================================================ REPLACE ID WITH VECTOR DICT ===================================================================###'''
#One hot encoding for each color id

t1 = np.array([i for i in range(9)])
encoder = LabelBinarizer()
t2 = encoder.fit_transform(t1)
t3 = []
for ele in t2:
  t3.append(tuple(ele))

vectortoid = dict(zip(t3, range(9)))
idtovector = dict(zip(range(9),t3))



#vectortoid[(0,0,0,0,1,0,0,0,0)],list(idtovector[7]) #use it like

In [0]:
'''###================================================ SET PARAMETERS HERE ==================================================###'''
patch_size = 4  #size of patch used for training cnn
stride = 4      #Stride used for creating above patches

n_epochs = 50   #Number of epoch in training
batch_size=128  #batch_size in training

alphawhite = 1  # (in range (1,INFINITY))  #parameter to control false positive of white class.Increase alphawhite if want to decrease false positive of white class.

In [0]:
'''###================================================ create_patches and create_lbls FUNCTON ==================================================###'''
def create_patches(img,patch_size,stride):   #FUNCTION TO CREATE PATCH FOR 4 BAND TRAINING AND TESTING IMAGES 
  nrow, ncol = img.shape[0],img.shape[1]
  i_startlist,i_endlist = np.arange(0, nrow-patch_size+1, stride),np.arange(0+patch_size, nrow, stride)
  j_startlist,j_endlist = np.arange(0, ncol-patch_size+1, stride),np.arange(0+patch_size, ncol, stride)
  patch_list = list()
  for istart, iend in zip(i_startlist, i_endlist):
      for jstart, jend in zip(j_startlist, j_endlist):
        patch_list.append(img[istart:iend, jstart:jend,:])
  patches1 = np.array(patch_list)
#   print("Input img shape:",img.shape,"Patches shape:",patches1.shape) #use if you want size patch and number of created patches
  return patches1


'''-----------------------------------------------------------------------------------------------------------------------------------------------'''


def create_lbls(mask,patch_size,stride):#FUNCTION TO CREATE PATCH AND HENCE MAXCOLOUR COLUR ONE HOT ENCODE, FOR 3 BAND TRAINING AND TESTING IMAGES 
  nrow, ncol = mask.shape[0],mask.shape[1]
  i_startlist,i_endlist = np.arange(0, nrow-patch_size+1, stride),np.arange(0+patch_size, nrow, stride)
  j_startlist,j_endlist = np.arange(0, ncol-patch_size+1, stride),np.arange(0+patch_size, ncol, stride)
  lbl_list = list()
  for istart, iend in zip(i_startlist, i_endlist):
      for jstart, jend in zip(j_startlist, j_endlist):
        
          lbl_list.append(list(idtovector[Counter(mask[istart:iend, jstart:jend].flatten()).most_common(1)[0][0]]))
  
  lbls1 = np.array(lbl_list)
#   print("Input img shape:",mask.shape,"lbls shape:",lbls1.shape)#use if you want size patch and number of created patches
  
  return lbls1
  
# patches = create_patches(x[0]) #USE THEM LIKE THIS
# lbls = create_lbls(ymask[0])   #USE THEM LIKE THIS

In [0]:
'''###================================ CREATE PATCHES FOR TRAINING AND TEST DATA, & LBLS FOR TRAINING DATA =========================================###'''

xpatches = []
test_images_patches = []
lblslist = []

for i,ele in enumerate(x):
  patches = create_patches(ele,patch_size,stride)
  xpatches.append(patches)
for ele in ymask:
  lbls = create_lbls(ele,patch_size,stride)
  lblslist.append(lbls)
for ele in test_images:
  patches = create_patches(ele,patch_size,stride)
  test_images_patches.append(patches)
  
xpatches = np.array(xpatches)
test_images_patches = np.array(test_images_patches)
lblslist = np.array(lblslist)

In [0]:
xpatches_ = xpatches[0]
lblslist_ = lblslist[0]
for i in range(1,nx):
  xpatches_ = np.concatenate((xpatches_, xpatches[i]), axis=0)
  lblslist_ = np.concatenate((lblslist_, lblslist[i]), axis=0)
# print(xpatches_.shape,lblslist_.shape)

# xpatches_    : Similar to x DATA FOR MODEL
# lblslist_    : Similar to Y DATA FOR MODEL

In [0]:
'''###================================================ CREATE MODEL FUNCTON ==================================================###'''
def create_model():  #FUNCTION TO CREATE KERAS MODEL
  
  #model
  model = Sequential()
  
  #Convolution layer 1
  model.add(Convolution2D(64, 1, 1, border_mode='same', input_shape=(patch_size, patch_size, 4), dim_ordering="tf"))
  model.add(BatchNormalization(mode=0))
  model.add(Activation("relu"))
  
  #pooling
  model.add(MaxPooling2D((2,2), dim_ordering="tf"))
  
  #Convolution layer 2
  model.add(Convolution2D(64, 1, 1, border_mode='same', dim_ordering="tf"))
  model.add(BatchNormalization(mode=0))
  model.add(Activation("relu"))
  
  #Convolution layer 3
  model.add(Convolution2D(128, 3, 3, border_mode='same', dim_ordering="tf"))
  model.add(BatchNormalization(mode=0))
  model.add(Activation("relu"))
  
  #pooling
  model.add(MaxPooling2D((2,2), dim_ordering="tf"))
  
  #fully connected laye
  model.add(Flatten())
  
  # output layer
  model.add(Dense(9,activation="softmax", name='output_layer'))

  model.compile(loss='categorical_crossentropy',optimizer= optimizers.Nadam(lr = 0.002),metrics=['accuracy'])
  model.summary()
  return model

In [0]:
'''###================================================ VISUALIZATION TRAINING FUNCTON ==================================================###'''
def visualize(hist):   #FUNCTION TO VISUALISE EPOCH DURING TRAINING
  
    temp_epochs = len(hist.history['loss'])
    
    # visualizing losses and accuracy
    train_loss=hist.history['loss']
    val_loss=hist.history['val_loss']
    train_acc=hist.history['acc']
    val_acc=hist.history['val_acc']
    xc=range(temp_epochs)



    plt.figure(1,figsize=(7,5))
    plt.plot(xc,train_loss)
    plt.plot(xc,val_loss)
    plt.xlabel('num of Epochs')
    plt.ylabel('loss')
    plt.title('train_loss vs val_loss')
    plt.grid(True)
    plt.legend(['train','val'])
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])
    plt.show()





    plt.figure(2,figsize=(7,5))
    plt.plot(xc,train_acc)
    plt.plot(xc,val_acc)
    plt.xlabel('num of Epochs')
    plt.ylabel('accuracy')
    plt.title('train_acc vs val_acc')
    plt.grid(True)
    plt.legend(['train','val'],loc=4)
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])
    plt.show()

In [0]:
'''###================================================ TRAINING FUNCTON ==================================================###'''

def train():           #FUNCTION TO TRAIN MODEL
  filename = 'modelsaved_once.h5'
    
  checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
  earlystop = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
  ReduceLROnPlateau_callback = ReduceLROnPlateau(monitor='val_acc', factor=0.5,patience=3,verbose=1, min_lr=0.0001, mode='auto')
  callbacks_list = [checkpoint,earlystop,ReduceLROnPlateau_callback]

      
  patches = xpatches_
  lbls = lblslist_
  model = create_model()

  xtrain, xval, ytrain, yval = train_test_split(patches, lbls,train_size=0.8,test_size=0.2,random_state=2,shuffle=True)
  t=time.time()
  hist = model.fit(xtrain, ytrain, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(xval, yval), callbacks=callbacks_list)
  print('Training time: %s' % (t - time.time()))

  model = load_model(filename)
  (loss, accuracy) = model.evaluate(xval, yval, batch_size=batch_size, verbose=1)
  print("\n[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

  visualize(hist)

  return model

In [0]:
'''###================================================ TRAINING in kfold FUNCTON ==================================================###'''

def train_kfold(n_folds):
  patches = xpatches_
  lbls = lblslist_
      
  skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=None)
  modellist = []
  for i, (train_idx, val_idx) in enumerate(skf.split(patches,lbls.argmax(1))):
      print("****************Running Fold ", i+1, "/", n_folds,"**************")
      
      
      filename = "modelsaved_kfold_"+str(i+1)+".h5"

      checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
      earlystop = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
      ReduceLROnPlateau_callback = ReduceLROnPlateau(monitor='val_acc', factor=0.5,patience=3,verbose=1, min_lr=0.0000001, mode='auto')
      callbacks_list = [checkpoint,earlystop,ReduceLROnPlateau_callback]
      
      
      xtrain, xval, ytrain, yval = patches[train_idx], patches[val_idx], lbls[train_idx], lbls[val_idx] #train_test_split(patches[], lbls,train_size=0.8,test_size=0.2,random_state=2,shuffle=True)  
      model = None  
      model = create_model()
      t=time.time()
      hist = model.fit(xtrain, ytrain, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(xval, yval), callbacks=callbacks_list)
      print('Training time: %s' % (t - time.time()))
      
      
      
      model = load_model(filename)
      (loss, accuracy) = model.evaluate(xval, yval, batch_size=batch_size, verbose=1)
      print("\n[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

      visualize(hist)
      modellist.append(modellist)
  return modellist

In [69]:
'''###=================================================================== Train without kfold cross validification(Quick) ================================================================###'''
model = train() # CALLING TRAIN FUNCTION

'###=================================================================== Train without kfold cross validification(Quick) ================================================================###'

In [0]:
'''###=================================================================== Train with kfold cross validification(Too much time taking) ================================================================###'''
n_folds = 2
modellist = train_kfold(n_folds) #Use this for kfold cross validation

In [0]:
# model = load_model('modelsaved_once.h5') #LOAD BEST MODEL SO FAR IF ABOVE TRAINING PROCESS IS TERMINATED DUE TO RUN TIME DIED

In [0]:
'''###================================================ PREDICT FUNCTON ==================================================###'''

def predict(model,index,sflag,pflag,alphawhite): #FUNCTION TO PREDICT THE OUTPUT
  
  '''
  index : index of image to predict from data
  sflag : take image from test dataset if sflag = "test" else if sflag = "train" then take from train dataset
  pflag : == "yes" if you want to visualise image ,else "no"
  alphawhite > 1: Increase alphawhite if want to decrease false positive of white class
  '''
  
  if sflag == "train":
    patches,real = xpatches[index],x[index]
  elif sflag == "test":
    patches,real = test_images_patches[index],test_images[index]
  else:
    print("WRANING : Wrong Input")
    return

  
  out = model.predict(patches,batch_size=32, verbose=False, steps=None)
  
  
  nrow, ncol = real.shape[0],real.shape[1]
  i_ = np.arange(0, nrow, patch_size)
  j_ = np.arange(0, ncol, patch_size)
  newimg = np.zeros((nrow,ncol,3),dtype = 'uint8')  #'uint8' !!! defalut is float64
  count = 0
  for istart, iend in zip(i_[:-1], i_[1:]):
      for jstart, jend in zip(j_[:-1], j_[1:]):
        twoele = heapq.nlargest(2, out[count])
        if twoele[0] == 0:
          if twoele[0]/twoele[1] > alphawhite :
            p = np.where(out[count] == twoele[0])[0][0]
          else:
            p = np.where(out[count] == twoele[1])[0][0]
        else:
          p = np.where(out[count] == twoele[0])[0][0]
        for i in range(0,patch_size):
          for j in range(0,patch_size):
            for k in range(3):
              newimg[istart+i,jstart+j,k] = idtopixel[p][k]
        count+=1
  
  if pflag == "yes":
    plot(newimg,"Predicted")
    plottiff(real,"Real Image")
    if sflag == "train":
      plot(y[index],"Given label")
    
       
  return (newimg,out)

In [0]:
'''###================================================ CALLING PREDICT FUNCTON ON TRAINING IMAGE ==================================================###'''
train_pred = []
for i in range(nx):
  (newimage,out) = predict(model,i,"train","yes",alphawhite)
  train_pred.append(newimage)

In [0]:
'''###================================================ CALLING PREDICT FUNCTON ON TESTING IMAGE ==================================================###'''
test_pred = []
for i in range(nytest):
  (newimage,out) = predict(model,i,"test","yes",alphawhite)
  test_pred.append(newimage)

In [0]:
#=============If using kfold then predict like this by each model combine and take average of output genrated by all models.=============
outlist = []
final_out = None
i = 0 #index of image to predict

for temp_model in modellist:
  (newimage,out) = predict(model,i,"test","no",alphawhite)
  outlist.append(out)
  
final_out = outlist[0] + outlist[1]  #ensembling the output of 0th and 1st model
# ==========================================================================================================================================






#================================= Ensembling model to one model if trained in kfold================================#
'''
def ensembleModels(modellist, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in modellist] 

    # averaging outputs
    yAvg=layers.average(yModels) 

    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns


model_input = Input(shape=(modellist[0]).input_shape[1:]) # c*h*w
modelEns = ensembleModels(modellist, model_input)
modelEns.summary()
'''

In [0]:
# model.save("drive/ML/sat_image_saved_model/modelsaved_once.h5")  #SAVE MODEL WEIGHT TO USE LATER

In [0]:
# =================================================== FUNCTION TO SMOOTHEN THE IMAGE FOR BETTER VISULISATION ========================================#
def smooth(real,startp,smooth_patch):
  realmask = np.zeros((real.shape[0],real.shape[1]))
  for i,j in product(range(real.shape[0]),range(real.shape[1])):
    realmask[i,j] = pixeltoid[tuple(real[i,j,:])]
  nrow, ncol = real.shape[0],real.shape[1]
  i_ = np.arange(startp, nrow, smooth_patch)
  j_ = np.arange(startp, ncol, smooth_patch)
  newimg = np.zeros((nrow,ncol,3),dtype = 'uint8')  #'uint8' !!! defalut is float64
  for istart, iend in zip(i_[:-1], i_[1:]):
      for jstart, jend in zip(j_[:-1], j_[1:]):
        for i in range(0,smooth_patch):
          for j in range(0,smooth_patch):
            for k in range(3):
              a = idtopixel[Counter(realmask[istart:iend, jstart:jend].flatten()).most_common(1)[0][0]][k]
              newimg[istart+i,jstart+j,k] = a
  return newimg

In [0]:
# =================================================== Printing smoothen function ========================================#

img = test_pred[0] #image to smoothen
startp = 1 # i.e i>=1 and j>=1 while smoothing image
smooth_patch = 2 # smoothening filter size
smoothen_png_image = smooth(img,startp,smooth_patch)

plot(smoothen_png_image,"Smoothen")

In [0]:
#============================================ Function to create 2d mask array of 3 band png labels array ======================================#
def ymasks(y):
  ymask1 = []
  for ele in y:
    temp = np.zeros((ele.shape[0],ele.shape[1]))
    for i,j in product(range(ele.shape[0]),range(ele.shape[1])):
      temp[i,j] = pixeltoid[tuple(ele[i,j,:])]
    ymask1.append(temp)
  return np.array(ymask)

In [0]:
#============================================ Function to calculate Confusion matrix, Confusion matrix report, Kappa coefficient and overallacc ======================================#

def accuracy(y,y_):  
  
  y_actu = ymasks(y)
  y_pred = ymasks(y_)
  
  y_actulist = []
  y_predlist = []
  

  for k in range(len(y_actu)):
    for ele in y_actu[k].flatten():
      y_actulist.append(ele)

    for ele in y_pred[k].flatten():
      y_predlist.append(ele)

  cm = confusion_matrix(y_actulist, y_predlist)
#   print(cm)

#   print(Counter(y_actulist)) #frequency of colour in y_actulist
#   print(Counter(y_predlist)) #frequency of colour in y_predlist

  target_names = []
  for j in range(max(len(Counter(y_actulist)),len(Counter(y_predlist)))):
    target_names.append(str(j))

  cm_report = classification_report(y_actulist, y_predlist, target_names=target_names)
#   print(cm_report)

  kappa = cohen_kappa_score(y_actulist, y_predlist, labels=None, weights=None, sample_weight=None)
  overallacc = accuracy_score(y_actulist, y_predlist,  normalize=True,sample_weight=None)
  print(kappa,overallacc)
  
  return (cm, cm_report,kappa,overallacc)

In [0]:
#============================================ Printing Confusion matrix, Confusion matrix report, Kappa coefficient and overallacc ======================================#
(cm, cm_report,kappa,overallacc) = accuracy(y,train_pred)

print("Confusion matrix:\n",cm)
print(" \nConfusion matrix report:\n",cm_report)
print("\nkappa coefficient:\n",kappa)
print("\noverallacc:\n",overallacc)